In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kerastuner as kt
import tensorflow as tf
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
train.head()

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
test.head()

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
sample_submission.head()

In [ ]:
train.drop('id', axis=1, inplace = True)
test.drop('id', axis=1, inplace = True)

In [ ]:
train['target']= train['target'].replace({'Class_1': 0, 'Class_2': 1, 'Class_3': 2, 'Class_4': 3, 'Class_5': 4, 'Class_6': 5, 'Class_7': 6, 'Class_8': 7,'Class_9': 8})

In [ ]:
X = train.iloc[:,:75]
y = train.iloc[:,75:]

In [ ]:
scale = StandardScaler()
X = scale.fit_transform(X)

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='selu', kernel_initializer = 'lecun_normal'))
    model.add(keras.layers.Dense(10, activation = 'softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(build_model, objective='val_accuracy',max_epochs=10, factor=3, directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
tuner.search(X, y, epochs=50, validation_split=0.2, callbacks=[stop_early])

In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X, y, epochs=20, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
history = hypermodel.fit(X, y, epochs=best_epoch, validation_split=0.2)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,7))
plt.grid(True)
plt.show()

In [ ]:
probability_predictions = hypermodel.predict(test)
probability_predictions

In [ ]:
sample_submission.iloc[:,1:10] = pd.DataFrame(probability_predictions)
sample_submission

In [ ]:
sample_submission.to_csv('submission.csv',index=False)